### Scikit-Learn: regresión lineal

En este notebook, entrenaremos un modelo simple de regresión lineal a partir de un dataset de rendimiento de estudiantes.

Se trata de uno de los modelos más simples, pero los pasos que se utilizan para entrenarlo son prácticamente idénticas para otros muchos modelos de aprendizaje supervisado (tanto de regresión, como de clasificación y clustering).

A partir de lo aprendido aquí, podríamos ejecutar otros modelos como _Regresión polinómica_, _SVM_ o _Random Forests_ inmediatamente sobre el mismo dataset (por supuesto, hay que tener cuidado con el _overfitting_!)

### Limpieza de datos

In [ ]:
# Primero, importamos todo lo que vamos a necesitar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

In [ ]:
# Y leemos el dataset
DATA_FOLDER = os.path.join("..", "data")
FILENAME = "student_performance.csv"
df = pd.read_csv(os.path.join(DATA_FOLDER, FILENAME))
df.dtypes

In [ ]:
df.describe()

In [ ]:
# La variable que queremos predecir es "Performance Index"
df['Performance Index'].value_counts().sort_index()

In [ ]:
# Existen variables que claramente ayudarán a predecir el performance index, como el número de horas de estudio
df['Hours Studied'].value_counts()


In [ ]:
# Extracurricular activities es una variable booleana (Si/No), aunque está en formato de string, luego veremos
# como tratarla
df['Extracurricular Activities'].value_counts()

## Exploración de los datos
- La variable ```Performance Index``` es la que queremos predecir (variable objetivo). Para ello, usaremos el resto de variables del dataset en la medida de lo posible.
- Vamos a hacer algunos plots para ver que variables podrían ser más interesantes a simple vista

In [ ]:
plt.scatter(df['Hours Studied'], df['Performance Index'])

In [ ]:
plt.scatter(df['Previous Scores'], df['Performance Index'])

In [ ]:
corr_data = df.drop("Extracurricular Activities", axis = 1)
correl = corr_data.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correl, annot=True, cmap="viridis", fmt=".2f", linewidths=0.5)
plt.tight_layout()
plt.show()

## Entrenamiento del modelo lineal
- Claramente, `Previous Score` es la columna más importante
- También hay algo de correlación con `Hours Studied`
- El resto de columnas no tienen prácticamente correlación, asi que no merece la pena incluirlas en un modelo lineal

In [ ]:
# En primer lugar, dividimos los datos en variables explicativas y variable objetivo (X e y)
X = df[['Hours Studied', 'Previous Scores']]
y = df["Performance Index"]
X.head()

In [ ]:
# Y hacemos un train-test split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# También es necesario escalar los datos para centrarlos en torno al 0, lo cuál mejora el rendimiento del modelo
# Haciendo que ninguna variable sea más importante que otras por su escala
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

In [ ]:
# Entrenamos el modelo
np.random.seed(42)
linear_reg = LinearRegression()
linear_reg.fit(x_train, y_train)

In [ ]:
# Y sacamos las predicciones para el conjunto de prueba
y_pred = linear_reg.predict(x_test)


In [ ]:
# Podemos calcular el R² para ver la precisión del modelo
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2:.4f}")

In [ ]:
# Y ahora podemos plottear los valores predecidos frente a los reales
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([min(y_test), max(y_test)], [min(y_pred), max(y_pred)], 'r--')
plt.xlabel("Valores reales")
plt.ylabel("Valores predecidos")
plt.show()

In [ ]:
# O un histograma de los residuos
residuals = y_pred - y_test
plt.hist(residuals, bins=30, edgecolor='black', alpha=0.7)
plt.xlabel("Residuos")
plt.ylabel("Frecuencia")
plt.show()

### Pipelines

- Podemos agilizar el proceso de entrenamiento usando `Pipeline`
- Especificamos todos los pasos y solo ejecutamos un `fit()` y un `predict()`
- Muy útil para procesos más complicados

In [ ]:
X = df[['Hours Studied', 'Previous Scores', 'Sleep Hours']]
y = df["Performance Index"]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
model = Pipeline([
    ("scaler", StandardScaler()),
    ("regressor", LinearRegression())
])

model.fit(x_train, y_train)
y_pred = model.predict(x_test)
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2:.4f}")

### Trabajar con datos categóricos

- Por último, vamos a entrenar un modelo usando también las otras variables y la variable categórica
- Para ello, usamos `pd.get_dummies`

In [ ]:
df = pd.get_dummies(df)
df

In [ ]:
from sklearn.linear_model import Lasso # Otro modelo lineal, con regularización añadida
X = df.drop(columns='Performance Index')
y = df["Performance Index"]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
model = Pipeline([
    ("scaler", StandardScaler()),
    ("regressor", Lasso())
])

model.fit(x_train, y_train)
y_pred = model.predict(x_test)
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2:.4f}")